In [6]:
from tensorflow.keras.models import load_model
import librosa
import numpy as np
import librosa.display
from pydub import AudioSegment
from pickle import load
import os

num_rows, num_columns = 40, 1292
max_pad_len = 1292
n_fft = 4096
n_hop = 1024
n_mfcc = 40

li = ['2.5톤', '5톤', '9.5톤', '10톤', '27톤',
      'K-1', 'K1a1', 'k10탄약운반차', 'K56', 'K77',
      'K288a1', 'K200', 'K800', 'km9ace', '교량전차',
      '다목적굴착기', '대형버스', '부식차', '살수차', '승용차',
      '장애물개척자', '통신가설차량', '화생방정찰차'] # 중분류, 소분류

def Class_ins(t):
      if t in (5, 6):
            return f'중분류 : 전차 소분류 : {li[t]}'
      elif t in (7, 8, 9, 10, 11, 12, 13, 14, 20, 22):
            return f'중분류 : 궤도/장갑차 소분류 : {li[t]}'
      elif t in (0, 1, 2, 3, 4, 15, 16, 17, 18, 19, 21):
            return f'중분류 : 차륜 전투차량 소분류 : {li[t]}'

def Result(filename):
    
    m4a_file = filename
    wav_filename = filename.split('.')[0] + '.wav'
    
    track = AudioSegment.from_file(m4a_file, format='m4a')
    file_handle = track.export(wav_filename, format='wav')
    
    # 1. 실무에 사용할 데이터 준비하기
    audio, sample_rate = librosa.load(file_handle, res_type='kaiser_fast', sr=44100, duration=30)
    
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc, hop_length=n_hop, n_fft=n_fft)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode="constant")

    mfccs = np.array(mfccs)
    
    # 2. 모델 불러오기
    model = load_model('../Saved_Model/Weight_best.hdf5')
    scaler = load(open('../Saved_Scale/minmax_scaler.pkl', 'rb'))
    
    x_test = scaler.transform(mfccs.reshape(1, num_rows * num_columns))

    predicted_classes = np.argmax(model.predict(x_test), axis=1)
    
    print(filename, Class_ins(int(predicted_classes)))

In [7]:
for filename in os.listdir('../file_upload'):
    Result('../file_upload/' + filename)

../file_upload/2,5t.m4a 중분류 : 차륜 전투차량 소분류 : 살수차
../file_upload/2.5톤1.m4a 중분류 : 차륜 전투차량 소분류 : 27톤
../file_upload/2.5톤2.m4a 중분류 : 차륜 전투차량 소분류 : 부식차
../file_upload/2.5톤3.m4a 중분류 : 차륜 전투차량 소분류 : 27톤
../file_upload/25t트럭.m4a 중분류 : 궤도/장갑차 소분류 : K77
../file_upload/2_5t(2).m4a 중분류 : 차륜 전투차량 소분류 : 9.5톤
../file_upload/2_5t(3).m4a 중분류 : 차륜 전투차량 소분류 : 9.5톤
../file_upload/2_5t(4).m4a 중분류 : 차륜 전투차량 소분류 : 살수차
../file_upload/2_5t(5).m4a 중분류 : 차륜 전투차량 소분류 : 살수차
../file_upload/2_5t.m4a 중분류 : 차륜 전투차량 소분류 : 27톤
../file_upload/4_5t 샵차.m4a 중분류 : 궤도/장갑차 소분류 : K288a1
../file_upload/800계열.m4a 중분류 : 궤도/장갑차 소분류 : K288a1
../file_upload/800장갑차.m4a 중분류 : 전차 소분류 : K-1
../file_upload/Amb.m4a 중분류 : 차륜 전투차량 소분류 : 27톤
../file_upload/k8080.m4a 중분류 : 차륜 전투차량 소분류 : 9.5톤
../file_upload/k8081.m4a 중분류 : 궤도/장갑차 소분류 : K288a1
../file_upload/k80810.m4a 중분류 : 궤도/장갑차 소분류 : K77
../file_upload/k80811.m4a 중분류 : 궤도/장갑차 소분류 : K288a1
../file_upload/k8082.m4a 중분류 : 궤도/장갑차 소분류 : 장애물개척자
../file_upload/k8083.m4a 중분류 : 차륜 전투차량 소분류 : 9.5톤
../f